### 1 - Articles dataset.

In [ ]:
import pyspark.sql.types as st
import pyspark.sql.functions as sf
from pyspark.sql import SparkSession, SQLContext

In [ ]:
# Spark Session app
spark = SparkSession \
  .builder \
  .appName('birdie-teste') \
  .getOrCreate()

# Create a SQL Context
sc = spark.sparkContext

In [ ]:
# Directory origem with all the pre-processing datas.
path_local_container = "/home/jovyan/work/"

# Add local diretory witl all files downloadeds the GDrive.
path_principal = path_local_container+"Documents/Projetos-Pessoais/testes-empresas/birdie/files/enwiki_latest_pages_articles_files/*"
path_category = path_local_container+"Documents/Projetos-Pessoais/testes-empresas/birdie/files/enwiki-category-sql/category.csv"

In [ ]:
# Extract schema
SCHEMA = spark.read.json(path_principal).schema

In [ ]:
# Load json.
df_original = spark \
    .read.json(path_principal, SCHEMA)

### 2 - Category dataset.

In [ ]:
read_category = spark \
    .read.option("delimiter","|") \
    .csv(path_category, header=True)

In [ ]:
# Romove equals duplicity registry in dataset
# Create and remove max field
read_category_distinct = read_category \
    .groupby("cat_id","cat_title","cat_pages","cat_subcats","cat_files") \
    .agg(sf.max("cat_id").alias("max")).drop("max")

In [ ]:
# Show if the dataset have still duplicitys
read_category_distinct\
    .groupby("cat_id")\
    .count()\
    .filter("count > 1")\
    .show()

In [ ]:
# Union the two datasets in one.
df_union_dataset = df_original \
    .join(read_category_distinct, df_original.id == read_category_distinct.cat_id, how='left')

In [ ]:
# Write dataset final in json
df_union_dataset \
    .write.mode("overwrite") \
    .json("/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final")

## ELSK

In [ ]:
# Please install ELSK in container
!pip install elasticsearch

In [1]:
# Import Elasticsearch package 
from elasticsearch import Elasticsearch, helpers
import os, json

In [2]:
# Connect to the elastic cluster
es = Elasticsearch([{'host':'172.19.0.3','port':9200}])

In [3]:
# test connection, if True its ok!
es.ping()

True

In [4]:
# Path with all json files.
directory = "/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/"

# while for the recover all the json files and broken each in individual payload per id
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        # recover all the json files
        with open(directory+filename, 'r') as open_file:
            print(directory+filename)
            # open the json files and broken each part in one
            data = [json.loads(line) for line in open(directory+filename).readlines()]
            
            # Index payload per payload in ELSK.
            helpers.bulk(es, data, index='birdie-wiki-final', doc_type='articles', request_timeout=40)


/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/part-00019-b8759e38-959f-4707-94b7-97074c29abc6-c000.json
/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/part-00036-b8759e38-959f-4707-94b7-97074c29abc6-c000.json
/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/part-00081-b8759e38-959f-4707-94b7-97074c29abc6-c000.json
/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/part-00007-b8759e38-959f-4707-94b7-97074c29abc6-c000.json
/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/part-00002-b8759e38-959f-4707-94b7-97074c29abc6-c000.json
/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/part-00140-b8759e38-959f-4707-94b7-97074c29abc6-c000.json
/home/jovyan/work/Documents/Projetos-Pessoais/testes-empresas/birdie/files/wiki_final/part-00119-b8759e38-959f-4707-94b7-97074c29abc6-c0

BulkIndexError: ('500 document(s) failed to index.', [{'index': {'_index': 'birdie-wiki-final', '_type': 'articles', '_id': 'uRhR_GoBH77kAisjAZa3', 'status': 403, 'error': {'type': 'cluster_block_exception', 'reason': 'blocked by: [FORBIDDEN/12/index read-only / allow delete (api)];'}, 'data': {'id': '1090966', 'text': 'Anglo-Spanish War (1585–1604)\n\nThe Anglo-Spanish War (1585–1604) was an intermittent conflict between the kingdoms of Spain and England that was never formally declared. The war was punctuated by widely separated battles, and began with England\'s military expedition in 1585 to what was then the Spanish Netherlands under the command of the Earl of Leicester in support of the resistance of the States General to Spanish Habsburg rule.\n\nThe English enjoyed some victories at Cádiz in 1587, and saw the Spanish Armada retreat in 1588, but then suffered severe defeats of the English Armada in 1589 and the Drake–Hawkins and Essex–Raleigh expeditions in 1595 and 1597 respect

#### Search with API

In [ ]:
res = es.search(index='birdie-wiki-final',body={'query':{'match_all':{}}})

print('Got %d hits:' %res['hits']['total'])

In [ ]:
res = es.search(index='birdie-wiki-final',body={'query':{}})

print(res['hits']['hits'])

In [ ]:
res = es.search(index='birdie-wiki-final',body={'query':{'match':{'id': 13809905}}})

print(res['hits']['hits'])

## Endpoints